<a href="https://colab.research.google.com/github/freshmandarina/Link-quality-estimation/blob/main/LQE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You can skip getting traces if you have the initial_dataset from my github page**

# Getting traces

In [ ]:
#1
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! ls

drive  sample_data


In [ ]:
#2
%cd /content/drive/MyDrive/IJS/project_folder/link-quality-estimation/

/content/drive/MyDrive/IJS/project_folder/link-quality-estimation


In [ ]:
#ze imam
#! git clone https://github.com/sensorlab/link-quality-estimation.git

Cloning into 'link-quality-estimation'...
remote: Enumerating objects: 5327, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 5327 (delta 10), reused 20 (delta 6), pack-reused 5298
Receiving objects: 100% (5327/5327), 197.50 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (162/162), done.
Checking out files: 100% (4156/4156), done.


In [ ]:
#3
!pip install -e .

In [ ]:
#4
!python setup.py install

In [ ]:
!find / -type d -name "dataset-1-rutgers_wifi"
#!rm -r /usr/local/lib/python3.7/dist-packages/LQE-0.1-py3.7.egg/output/datasets/dataset-1-rutgers_wifi
#!python ./datasets/trace1_Rutgers/transform.py

find: ‘/proc/27/task/27/net’: Invalid argument
find: ‘/proc/27/net’: Invalid argument
find: ‘/proc/227/task/227/net’: Invalid argument
find: ‘/proc/227/net’: Invalid argument


In [ ]:
from datasets.trace1_Rutgers.transform import get_traces
#traces = list(get_traces())

#My definitions

## basic

In [37]:
from typing import List, Tuple, Generator, Iterator
from enum import Enum
from os import path

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

# Set custom matplotlib themes
#try:
#    plt.style.use(['science', 'ieee'])
#except OSError:
#    pass

from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, MinMaxScaler
from sklearn.ensemble import IsolationForest, BaggingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM, SVC
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, ParameterGrid

from sklearn import metrics, preprocessing, ensemble, linear_model, model_selection, neighbors, pipeline, svm, base


#from datasets.trace1_Rutgers.transform import get_traces, dtypes, get_traces2
#from datasets.trace1_Rutgers import NOISE_SOURCES

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, backend as K, callbacks

#my addons
from random import seed, randint
from itertools import compress

tf.config.set_soft_device_placement(True)

import joblib
from joblib import Parallel, delayed

from pandas import DataFrame

In [38]:
def give_start_stop_length():
    rng = np.random.default_rng()
    seed(rng.choice(1000))

    START = randint(0, 200)
    LENGTH = randint(50, 200)
    if 300 < START + LENGTH:
      over = START + LENGTH - 300
      START = START - over
    STOP = START + LENGTH

    if STOP >= 300:
      START, STOP, LENGTH = give_start_stop_length()

    return START, STOP, LENGTH

In [39]:
SEED = 0xDEADBEEF

NUM_OF_SAMP = 2123
TX_POWER = 10.0 # dBm
RSSI_BASE = -95.0 # dBm; not sure for this one. Most of the Atheros' have -95
FREQ = 2450.0 # MHz


In [40]:
def name2coords(name: str) -> Tuple[float, float]:
    """Name of the node contains information about it's relative position in testbed"""
    part = name[len('node'):] # extract only numeric part 'node3-2' -> '3-2'
    coords = [float(i) for i in part.split('-')]
    return tuple(coords)


In [41]:
def get_unique_pairs(df = None, i = 0):
    #get cut
    if i == 0:
      df = get_cut_dataset()

    pairs = df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    return pairs

In [42]:
def get_initial_dataset() -> List[pd.DataFrame]:
    
    return initial_dataset
    #old code
    def process(df: pd.DataFrame):
        assert len(df.index) == 300

        if df.received.sum() != 300:
            return None

        df['src'] = df['src'].apply(name2coords) # str -> coordinates
        df['dst'] = df['dst'].apply(name2coords) # str -> coordinates
        df['seq'] = df.index # preserve sequence number
        df['rss'] = df.rssi + RSSI_BASE # received signal strength

        # Fix #1: Initial values are abnormal on every link. Replace them to avoid misclassification
        mean, stdev = np.mean(df[df.seq > 3].rss), np.std(df[df.seq > 3].rss)
        df.loc[df.seq < 3, 'rss'] = np.random.normal(mean, stdev, size=3)

        return df

    # Parallel link transformation
    traces = get_traces2(n_jobs=-1)
    dfs = Parallel(n_jobs=-1)(delayed(process)(df) for df in traces)

    # Merge them at the end
    output = pd.concat(dfs, ignore_index=True)
    return output

In [43]:
def get_cut_dataset() -> List[pd.DataFrame]:
    df = get_initial_dataset() 

    #geting the parameters for cutting dataframe
    x_start = START
    x_stop = STOP

    #geting the df to start from START index and end on END index. We cut the rest
    new_df = []
    new_df = pd.DataFrame(new_df)
    for _ in range(NUM_OF_SAMP):
      temp = new_df
      new_df = df[x_start:x_stop]
      x_start = x_start + 300
      x_stop = x_stop + 300
      new_df = temp.append(new_df,ignore_index=True)
    new_df = repair_seq_cut(new_df)

    return new_df

In [44]:
def repair_seq_cut(df):
  
    #user to repair seq numbers for cut dataset
    df = df.drop(labels=['seq'], axis=1)
    seq = pd.DataFrame([])
    for n in range(2123):
      temp = seq
      seq = pd.DataFrame({ 'seq' : range(1, LENGTH+1)}).astype(int)
      seq = temp.append(seq,ignore_index=True)
    df['seq'] = seq
    return df

In [45]:
def give_randomizing_array(i=0):

    #used to get the START STOP and LENGTH for each sample in our dataset
    array = np.empty((NUM_OF_SAMP, 3))

    x_start = START
    x_stop = STOP
    x_length = LENGTH

    if i == 1:
      for n in range(NUM_OF_SAMP):
        x_start, x_stop, x_length = give_start_stop_length()
        array[n][0] = x_start
        array[n][1] = x_stop
        array[n][2] = x_length
    if i == 0:
      for n in range(NUM_OF_SAMP):
        array[n][0] = x_start
        array[n][1] = x_stop
        array[n][2] = x_length
  
    return array
    

In [46]:
def get_random_dataset(i=0):

    #used to cut every dataset sample into a random length
    rng = np.random.default_rng()

    df = get_initial_dataset()

    lengths = pd.DataFrame([])
    new_df = pd.DataFrame([])
    array = give_randomizing_array(i).astype(int)

    for n in range(NUM_OF_SAMP):
      
      x_start = array[n][0] + 300*n
      x_stop = array[n][1] + 300*n
      x = [array[n][2]]*array[n][2]
      lengths = lengths.append(x,ignore_index=True)

      temp = new_df
      new_df = df[x_start:x_stop]
      new_df = temp.append(new_df,ignore_index=True)

    new_df['length'] = lengths
    new_df = repair_seq(new_df,array)
    return new_df, array

In [47]:
def repair_seq(df,array):

    #used to repair seq numbers for cut dataset
    df = df.drop(labels=['seq'], axis=1)
    seq = pd.DataFrame([])
    for n in range(2123):
      temp = seq
      seq = pd.DataFrame({ 'seq' : range(1, array[n][2]+1)}).astype(int)
      seq = temp.append(seq,ignore_index=True)
    df['seq'] = seq
    return df

In [48]:
def check_line(affected_links):
    #this is so we can get rid of duplicates
    check_line = []
    check = 0
    for n in range(len(affected_links)):
      for m in range(len(check_line)):
        if check_line[m] == affected_links[n]:
          check = 1
      if check == 0:
        check_line.append(affected_links[n])
      check = 0

    return check_line

## cut/uncut - spikes, norecovery, recovery, slow

In [49]:
def norecovery_anomaly_injector_cut(scaler=None, random_state=None) -> pd.DataFrame:
    # Get ready-to-use prepared dataset for anomaly injection
    df = get_cut_dataset()
    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs() #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 1
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    # No-recovery anomaly has randomized start, which will appear randomly between 2/3 (originaly 200th) and 14/15 (o. 280th) of a sample
    # Let's assume there is no more the 1/3 of malicious samples.
    randomize_start = lambda: rng.integers(round((2*LENGTH)/3), round((14*LENGTH)/15), endpoint=True)
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        assert size == LENGTH, f'something is wrong. Size {size} is not {LENGTH}'

        start = randomize_start()
        query = query & (df.seq >= start)
        df.loc[query, 'anomaly'] = True

        ## Now inject actual annomaly

        # get delta between median and noise floor.
        delta = abs(RSSI_BASE - df.loc[query, 'rss'].median())

        # Drop should be around 30 or less, depending on delta
        delta = delta if delta < 30 else (30 - rng.uniform(0, 3))

        # Apply drop where anomaly should have appeared
        df.loc[query & df.anomaly, 'rss'] -= delta

        # Fix values below noise floor
        df.loc[query & df.anomaly & (df['rss'] < RSSI_BASE), 'rss'] = RSSI_BASE

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df

In [50]:
def recovery_anomaly_injector_cut(scaler=None, random_state=None) -> pd.DataFrame:
    # Get ready-to-use prepared dataset for anomaly injection
    df = get_cut_dataset()

    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs() #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 2
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    # Random drop starts between 1/12 (o. 25th) and 11/12(o. 275th) packet
    randomize_start = lambda: rng.integers(round(LENGTH/12), round((11*LENGTH)/12), endpoint=True)

    # Randomly recovers after 3th to 1/15 (o. 20th) packets
    randomize_length = lambda: rng.integers(3, round(LENGTH/15), endpoint=True)

    # Randomly select affected links
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        assert size == LENGTH, f'something is wrong. Size {size} is not {LENGTH}'

        start = randomize_start()
        stop = start + randomize_length()
        query = query & (df.seq >= start) & (df.seq <= stop)
        df.loc[query, 'anomaly'] = True

        ## Now actually inject anomalies

        # get delta between median and noise floor.
        delta = abs(RSSI_BASE - df.loc[query, 'rss'].median())

        # Drop should be around 30 or less, depending on delta
        delta = delta if delta < 30 else (30 - rng.uniform(0, 3))

        # Apply drop where anomaly should have appeared
        df.loc[query & df.anomaly, 'rss'] -= delta

        # Fix values below noise floor
        df.loc[query & df.anomaly & (df['rss'] < RSSI_BASE), 'rss'] = RSSI_BASE

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df


In [51]:
from pandas.core.indexers import length_of_indexer
def spike_anomaly_injector_cut(scaler=None, random_state=None) -> pd.DataFrame:

    # Get cut-ready-to-use prepared dataset for anomaly injection
    df = get_cut_dataset()
    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs() #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 3
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    # The probability for spike anomaly to appear is:
    anomaly_probability = 3. / LENGTH

    # Randomly select links, which will be altered
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        assert size == LENGTH, f'something is wrong. Size {size} is not {LENGTH}'
        df.loc[query, 'anomaly'] = np.random.choice([False, True], p=[1. - anomaly_probability, anomaly_probability], size=LENGTH)


        ## Now inject actual annomaly

        # get delta between median and noise floor.
        delta = abs(RSSI_BASE - df.loc[query, 'rss'].median())

        # Drop should be around 30 or less, depending on delta
        delta = delta if delta < 30 else (30 - rng.uniform(0, 3))

        # Apply drop where anomaly should have appeared
        df.loc[query & df.anomaly, 'rss'] -= delta

        # Fix values below noise floor
        df.loc[query & df.anomaly & (df['rss'] < RSSI_BASE), 'rss'] = RSSI_BASE

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df

In [52]:
def slow_anomaly_injector_cut(scaler=None, random_state=None) -> pd.DataFrame:
    # Get ready-to-use prepared dataset for anomaly injection
    df = get_cut_dataset()

    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs() #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 4
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    # randomize start of slow degradation between LENGTH/30 (o. 10th) and LENGTH/3 (o. 200th) packet
    rand_start = lambda: rng.integers(0, round(LENGTH/15), endpoint=True)
    rand_duration = lambda: rng.integers(round(LENGTH/2), round((14*LENGTH)/15), endpoint=True)
    rand_rate = lambda: rng.uniform(0.5, 1.5, size=LENGTH) / -LENGTH

    def slope(seq, rate, start):
        # General curve
        curve = rate * (seq - start)
        curve[curve > 0] = 0  # Correction #1: Remove values above 0
        #curve[curve < (rate * (end - start))] = rate * (end - start) # Correction #2: stop falling after "end" value

        # Sanity checks
        assert np.all(rate < 0), f'Rate should have been negative. Got {rate}'

        return curve

    # Randomly select affected links
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        assert size == LENGTH, f'something is wrong. Size {size} is not {LENGTH}'

        start = rand_start()
        end = start + rand_duration()
        rate = rand_rate()

        df.loc[query & (df.seq >= start), 'anomaly'] = True
        df.loc[query, 'rss'] += slope(seq=df.loc[query].seq, rate=rate, start=start)
        #df.loc[query, 'rss'] += factor(seq=df.loc[query,:].seq, offset=start)

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df


In [53]:
def get_all_cut():

  df1 = norecovery_anomaly_injector_cut()
  df2 = recovery_anomaly_injector_cut()
  df3 = spike_anomaly_injector_cut()
  df4 = slow_anomaly_injector_cut()

  df = df1.append(df2).append(df3).append(df4,ignore_index=True)

  np_rss = df['rss'].to_numpy()
  np_anomaly = df['anomaly'].to_numpy()
  np_type = df['type'].to_numpy()

  #cut anomaly and type
  anomaly_np = []
  t = 0
  for n in range(NUM_OF_SAMP*4):
    for i in range(LENGTH):
      t=0
      if np_anomaly[LENGTH*n + i] == True:
        anomaly_np.append(True)
        t=1
        break
    if t == 0:
      anomaly_np.append(False)

  type_np = []
  for n in range(len(anomaly_np)):
    if anomaly_np[n] == True:
      type_np.append(np_type[LENGTH*n])
    else:
      type_np.append(0)
    
  #get mask
  mask_np = []
  for n in range(len(np_anomaly)):
    if np_anomaly[n] == True:
      mask_np.append(1)
    else:
      mask_np.append(0)


  #append everything
  dataset_np = np.empty((NUM_OF_SAMP*4, LENGTH*2 +2))
  for n in range(NUM_OF_SAMP*4):
    np_rss_temp = np_rss[LENGTH*n:LENGTH*(n+1)]
    np_rss_temp = np.append(np_rss_temp,anomaly_np[n])
    np_rss_temp = np.append(np_rss_temp,type_np[n])
    dataset_np[n] = np.append(np_rss_temp,mask_np[LENGTH*n:LENGTH*(n+1)])

  dataset_df = pd.DataFrame(dataset_np)
  dataset_df = pd.DataFrame.rename(dataset_df, columns={LENGTH : "anomaly"})
  dataset_df = pd.DataFrame.rename(dataset_df, columns={(LENGTH+1) : "type"})

  #del dataset_df['Unnamed: 0']

  return dataset_df


## random - spikes, norecovery, recovery, slow

In [54]:
def norecovery_anomaly_injector(scaler=None, random_state=None) -> pd.DataFrame:
    # Get ready-to-use prepared dataset for anomaly injection
    df, lengths_original = get_random_dataset(1)

    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs(df, 1) #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)
    
    
    df['anomaly'] = False
    df['type']= 1
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    # No-recovery anomaly has randomized start, which will appear randomly between 2/3 (originaly 200th) and 14/15 (o. 280th) of a sample
    # Let's assume there is no more the 1/3 of malicious samples.
    
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:

        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        randomize_start = lambda: rng.integers(round((2*size)/3), round((14*size)/15), endpoint=True)
        
        # return df,query,size,temp_length
        # assert size == temp_length, f'something is wrong. Size {size} is not {temp_length}'

        start = randomize_start()
        query = query & (df.seq >= start)
        df.loc[query, 'anomaly'] = True

        ## Now inject actual annomaly

        # get delta between median and noise floor.
        delta = abs(RSSI_BASE - df.loc[query, 'rss'].median())

        # Drop should be around 30 or less, depending on delta
        delta = delta if delta < 30 else (30 - rng.uniform(0, 3))

        # Apply drop where anomaly should have appeared
        df.loc[query & df.anomaly, 'rss'] -= delta

        # Fix values below noise floor
        df.loc[query & df.anomaly & (df['rss'] < RSSI_BASE), 'rss'] = RSSI_BASE

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df, lengths_original

In [55]:
def recovery_anomaly_injector(scaler=None, random_state=None) -> pd.DataFrame:
    # Get ready-to-use prepared dataset for anomaly injection
    df, lengths_original = get_random_dataset(1)

    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs(df, 1) #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 2
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    # Randomly select affected links
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        # Random drop starts between 1/12 (o. 25th) and 11/12(o. 275th) packet
        randomize_start = lambda: rng.integers(round(size/12), round((11*size)/12), endpoint=True)

        # Randomly recovers after 3th to 1/15 (o. 20th) packets
        randomize_length = lambda: rng.integers(3, round(size/15), endpoint=True)

        start = randomize_start()
        stop = start + randomize_length()
        query = query & (df.seq >= start) & (df.seq <= stop)
        df.loc[query, 'anomaly'] = True

        ## Now actually inject anomalies

        # get delta between median and noise floor.
        delta = abs(RSSI_BASE - df.loc[query, 'rss'].median())

        # Drop should be around 30 or less, depending on delta
        delta = delta if delta < 30 else (30 - rng.uniform(0, 3))

        # Apply drop where anomaly should have appeared
        df.loc[query & df.anomaly, 'rss'] -= delta

        # Fix values below noise floor
        df.loc[query & df.anomaly & (df['rss'] < RSSI_BASE), 'rss'] = RSSI_BASE

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df, lengths_original


In [56]:
from pandas.core.indexers import length_of_indexer
def spike_anomaly_injector(scaler=None, random_state=None) -> pd.DataFrame:

    # Get cut-ready-to-use prepared dataset for anomaly injection
    df, lengths_original = get_random_dataset(1)

    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs(df, 1) #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 3
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)
    

    # Randomly select links, which will be altered
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)

    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        # The probability for spike anomaly to appear is:
        anomaly_probability = 3. / size
        df.loc[query, 'anomaly'] = np.random.choice([False, True], p=[1. - anomaly_probability, anomaly_probability], size=size)

        ## Now inject actual annomaly

        # get delta between median and noise floor.
        delta = abs(RSSI_BASE - df.loc[query, 'rss'].median())

        # Drop should be around 30 or less, depending on delta
        delta = delta if delta < 30 else (30 - rng.uniform(0, 3))

        # Apply drop where anomaly should have appeared
        df.loc[query & df.anomaly, 'rss'] -= delta

        # Fix values below noise floor
        df.loc[query & df.anomaly & (df['rss'] < RSSI_BASE), 'rss'] = RSSI_BASE

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df, lengths_original

In [57]:
def slow_anomaly_injector(scaler=None, random_state=None) -> pd.DataFrame:
    # Get ready-to-use prepared dataset for anomaly injection
    df, lengths_original = get_random_dataset(1)

    # Random number generator
    rng = np.random.default_rng(random_state)

    # Now, find unique pairs
    unique_pairs = get_unique_pairs(df, 1) #df.apply(lambda row: (row['src'], row['dst'], row['noise']), axis=1).unique()
    n_pairs = len(unique_pairs)

    df['anomaly'] = False
    df['type']= 4
    df['original'] = df.rss

    # approx. 33.3% of links will have anomaly
    n_anomaly_pairs = int(n_pairs // 3)

    def slope(seq, rate, start):
        # General curve
        curve = rate * (seq - start)
        curve[curve > 0] = 0  # Correction #1: Remove values above 0
        #curve[curve < (rate * (end - start))] = rate * (end - start) # Correction #2: stop falling after "end" value

        # Sanity checks
        assert np.all(rate < 0), f'Rate should have been negative. Got {rate}'

        return curve

    # Randomly select affected links
    affected_links = rng.choice(unique_pairs, n_anomaly_pairs)
    affected_links = check_line(affected_links)
    
    for src, dst, noise in affected_links:
        query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
        size = len(df[query].index)
        # randomize start of slow degradation between LENGTH/30 (o. 10th) and LENGTH/3 (o. 200th) packet
        rand_start = lambda: rng.integers(0, round(size/15), endpoint=True)
        rand_duration = lambda: rng.integers(round(size/2), round((14*size)/15), endpoint=True)
        rand_rate = lambda: rng.uniform(0.5, 1.5, size=size) / -size


        start = rand_start()
        end = start + rand_duration()
        rate = rand_rate()

        df.loc[query & (df.seq >= start), 'anomaly'] = True
        df.loc[query, 'rss'] += slope(seq=df.loc[query].seq, rate=rate, start=start)
        #df.loc[query, 'rss'] += factor(seq=df.loc[query,:].seq, offset=start)

    assert np.any(df.original != df.rss)

    # Scale links afterwards
    if scaler:
        for src, dst, noise in unique_pairs:
            query = (df.src==src) & (df.dst==dst) & (df.noise==noise)
            df.loc[query, ['rss']] = scaler.fit_transform(df.loc[query, ['rss']])

    assert len(df.index) != 0
    return df, lengths_original


In [58]:
def get_all():

  df1, lengths_original1 = norecovery_anomaly_injector()
  df2, lengths_original2 = recovery_anomaly_injector()
  df3, lengths_original3 = spike_anomaly_injector()
  df4, lengths_original4 = slow_anomaly_injector()

  lo1 = pd.DataFrame(lengths_original1)[2]
  lo2 = pd.DataFrame(lengths_original2)[2]
  lo3 = pd.DataFrame(lengths_original3)[2]
  lo4 = pd.DataFrame(lengths_original4)[2]

  lo = lo1.append(lo2).append(lo3).append(lo4,ignore_index=True)
  df = df1.append(df2).append(df3).append(df4,ignore_index=True)

  np_rss = df['rss'].to_numpy()
  np_anomaly = df['anomaly'].to_numpy()
  np_type = df['type'].to_numpy()

  #and anomaly 
  anomaly_np = []
  temp_len = 0
  t = 0
  for n in range(NUM_OF_SAMP*4):
    for i in range(lo[n]):
      t=0
      if np_anomaly[temp_len + i] == True:
        anomaly_np.append(True)
        t=1
        break
    if t == 0:
      anomaly_np.append(False)

    temp_len = lo[n] + temp_len
  anomaly_df = pd.DataFrame(anomaly_np)

  #and type
  type_np = []
  temp_len = 0
  for n in range(NUM_OF_SAMP*4):
    if anomaly_np[n] == True:
      type_np.append(np_type[temp_len])
    else:
      type_np.append(0)
      
    temp_len = lo[n] + temp_len
  type_df = pd.DataFrame(type_np)

  #creating a array with properties
  properties = pd.DataFrame([])
  properties['length'] = lo
  properties['anomaly'] = anomaly_df
  properties['type'] = type_df

  properties = properties.to_numpy()

  #get mask
  mask_np = []
  for n in range(len(np_anomaly)):
    if np_anomaly[n] == True:
      mask_np.append(1)
    else:
      mask_np.append(0)
  np_mask = np.array(mask_np)
  mask_df = pd.DataFrame(mask_np)
  #reshape rss

  temp = 0
  rss = np.empty((NUM_OF_SAMP*4, 1), dtype=object)
  rss = rss.reshape((NUM_OF_SAMP*4, ))
  for n in range(NUM_OF_SAMP*4):
    rss[n] = np_rss[temp:temp+lo[n]]
    temp = temp + lo[n]

  #reshape mask
  temp = 0
  mask = np.empty((NUM_OF_SAMP*4, 1), dtype=object)
  mask = mask.reshape((NUM_OF_SAMP*4, ))
  for n in range(NUM_OF_SAMP*4):
    mask[n] = np_mask[temp:temp+lo[n]]
    temp = temp + lo[n]


  return properties, rss, mask


# Setting START, STOP, LENGTH for cut/uncut and download the initial_dataset

For uncut

In [ ]:
#for uncut (original length of 300)
START = 0
STOP = 300
LENGTH = 300

For cut (so all samples will be of equal cut length)

In [1]:
#for cut
START, STOP, LENGTH = give_start_stop_length()

For the random dataset you dont have to set it up

Getting your initial dataset so we dont need to get traces 

In [ ]:
import pandas as pd
import shutil
import pathlib
import os
from google.colab import drive
drive.mount('/content/drive')
#set your own path to file given in github (i had it on my drive)
shutil.copy("/content/drive/MyDrive/initial_dataset.csv","/content")

initial_dataset = pd.read_csv('initial_dataset.csv',delim_whitespace = False, header = 0)
del initial_dataset['Unnamed: 0']

# Getting your dataset

## cut/uncut

In [ ]:
#run this to get dataset for cut/uncut (depends on what you chose it the "Setting START, STOP, LENGTH ...")
dataset = get_all_cut()

In [ ]:
#upload csv
dataset.to_csv("dataset.csv")

## random (every sample has its own length)

In [ ]:
#run this to get dataset for random dataset (consists of properties, rss, and mask)
properties, rss, mask = get_all()

In [ ]:
np.savez("dataset_properties",properties)
np.savez("dataset_rss",rss)
np.savez("dataset_maks",mask)